In [1]:
#Siamese Network on MNIST

In [2]:
#imports
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf 
from tensorflow.examples.tutorials.mnist import input_data # for data
import tensorflow as tf
import numpy as np
import os

In [3]:
#importing mnist dataset as a tf dataset
mnist = input_data.read_data_sets("../Data/MNIST/", one_hot=False)

Extracting ../Data/MNIST/train-images-idx3-ubyte.gz
Extracting ../Data/MNIST/train-labels-idx1-ubyte.gz
Extracting ../Data/MNIST/t10k-images-idx3-ubyte.gz
Extracting ../Data/MNIST/t10k-labels-idx1-ubyte.gz


In [4]:
#functions to set up the model
def twin_net(x):
    '''set up one instance of twin network
                1024 -> 1024 -> 2       '''
    #layer 1
    fc1 = make_fc(x, 1024, "fc1")
    activ1 = tf.nn.relu(fc1)
    #layer2
    fc2 = make_fc(activ1, 1024, "fc2")
    activ2 = tf.nn.relu(fc2)
     #layer3
    fc3 = make_fc(activ2, 2, "fc3")
    
    return fc3

#create an affine layer
def make_fc(bottom, n_weight, name):
    '''return speccified fc layer w/ correct sizing, normal weights, constant bias'''
    prev_shape = bottom.get_shape()[1]
    W = tf.get_variable(name=name+'_W', dtype=tf.float32, shape=[prev_shape, n_weight], initializer=tf.truncated_normal_initializer(stddev=0.01))
    b = tf.get_variable(name=name+'_b', dtype=tf.float32, initializer=tf.constant(value=0.01, shape=[n_weight], dtype=tf.float32))
    fc = tf.nn.bias_add(tf.matmul(bottom, W), b)
    
    return fc
    
def contrastive_loss():
    '''spring-like loss function comparing output from twin nets
    --> on learned manifold, similar inputs map to nearby points'''
    #pos loss: (1/2N) * SUM[1toN](y)* d^2 [ <-- summation is eucd^2]
    labels_t = y_  #bool: do labels match 
    eucd2 = tf.reduce_sum(tf.pow(tf.sub(o1, o2), 2), 1) #squared eucd between nets
        #loss:
        
    print(labels_t)
    print('labels, then eucd2')
    print(eucd2)
    
    pos = tf.mul(labels_t, eucd2)

    #neg loss: (1-y)*max(margin-d, 0)^2
    labels_f = tf.sub(1., y_) #1-y
    eucd = tf.sqrt(eucd2+1e-6)
    margin = 5.0
    C = tf.constant(value=margin)
        #loss:
    neg = tf.mul(labels_f, tf.pow(tf.maximum(tf.sub(C, eucd), 0), 2))
    
    #full loss:
    loss = tf.reduce_mean(tf.add(pos, neg))
    return loss

In [5]:
#set up the model graph structure (connect nets)
'''sets up entire siamese net (including twin nets and loss)'''
x1 = tf.placeholder(tf.float32, [None, 784])
x2 = tf.placeholder(tf.float32, [None, 784])

with tf.variable_scope("siamese") as scope:
    o1 = twin_net(x1)
    scope.reuse_variables()
    o2 = twin_net(x2)

y_ = tf.placeholder(tf.float32, shape=[None])
loss = contrastive_loss()
train_step = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

Tensor("Placeholder_2:0", shape=(?,), dtype=float32)
labels, then eucd2
Tensor("Sum:0", shape=(?,), dtype=float32)


In [6]:
#execution of graph
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    #run model:
    for it in range(10000):
        batch_x1, batch_y1 = mnist.train.next_batch(128) #net 1
        batch_x2, batch_y2 = mnist.train.next_batch(128) #net 2
        batch_y = (batch_y1 == batch_y2).astype('float') #comparison for contrastive loss
              
        _, loss_v = sess.run([train_step, loss], feed_dict ={
                x1:batch_x1, 
                x2:batch_x2, 
                y_:batch_y})
    
        #printing intermediate results:
        if (it % 10 == 0): 
            print("step: ", it, " loss: ", loss_v)

step:  0  loss:  22.029
step:  10  loss:  8.10551
step:  20  loss:  2.61952
step:  30  loss:  2.61668
step:  40  loss:  2.59073
step:  50  loss:  3.90316
step:  60  loss:  2.71831
step:  70  loss:  1.88482
step:  80  loss:  1.99286
step:  90  loss:  3.55095
step:  100  loss:  1.54913
step:  110  loss:  3.07055
step:  120  loss:  1.97322
step:  130  loss:  6.01779
step:  140  loss:  1.49061
step:  150  loss:  1.81153
step:  160  loss:  2.42218
step:  170  loss:  2.11591
step:  180  loss:  2.35878
step:  190  loss:  3.22625
step:  200  loss:  1.7571
step:  210  loss:  1.99928
step:  220  loss:  3.22604
step:  230  loss:  2.15497
step:  240  loss:  1.97825
step:  250  loss:  1.72006
step:  260  loss:  1.59874
step:  270  loss:  1.51805
step:  280  loss:  1.88649
step:  290  loss:  1.69565
step:  300  loss:  2.02736
step:  310  loss:  1.77043
step:  320  loss:  1.80751
step:  330  loss:  1.21554
step:  340  loss:  1.60711
step:  350  loss:  1.02781
step:  360  loss:  1.36096
step:  370  lo

KeyboardInterrupt: 